In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from math import floor
import os
import torchvision.models

In [3]:
import torchvision.models as models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vggnet = models.vgg19(pretrained=True)

C:\Users\hdape\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hdape\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
train_path = 'DataSet/VGG2_Gen2_NewAug_V3/Training'
pokemonlist = os.listdir('DataSet/VGG2_Gen2_NewAug_V3/Training')
data_transforms = transforms.Compose([
                    transforms.Resize((224,224)),
                    transforms.ToTensor()])
train_datasets = torchvision.datasets.ImageFolder(root = train_path, transform = data_transforms)

hyperparameters = [1,1]

train_loader = torch.utils.data.DataLoader(train_datasets, batch_size=hyperparameters[0], num_workers=hyperparameters[1], shuffle=True, collate_fn=None, pin_memory=False)
def features_to_folder(loader, path):
    n = 0
    for img, label in loader:
      features = vggnet.features(img)

      features_tensor = torch.from_numpy(features.detach().numpy())
      x = torch.tensor(label)
      print(features_tensor.shape)
      print('working on ' + str(pokemonList[x.item()]))
      # show the overall progress
      print('working on ' + str(n) + ' out of ' + str(len(train_datasets)))

      q = x.item()
      folder_name = path + '/' + str(pokemonList[q])
      if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
      torch.save(features_tensor.squeeze(0), folder_name + '/' + str(n) + '.tensor')
      n += 1

features_to_folder(train_loader, train_path)

def test(loader, path):
  for img, label in loader:
      features = vggnet.features(img)
      features_tensor = torch.from_numpy(features.detach().numpy())
      x = torch.tensor(label)
      if features_tensor.shape[1] != 512:
        print(features_tensor.shape, pokemonList[x.item()])
#test(train_loader, train_path)

C:\Users\hdape\AppData\Local\Temp\ipykernel_31620\1077587920.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(label)


torch.Size([1, 512, 7, 7])
working on Sentret
working on 0 out of 19920
torch.Size([1, 512, 7, 7])
working on Blastoise
working on 1 out of 19920
torch.Size([1, 512, 7, 7])
working on Entei
working on 2 out of 19920
torch.Size([1, 512, 7, 7])
working on Charmander
working on 3 out of 19920
torch.Size([1, 512, 7, 7])
working on Skiploom
working on 4 out of 19920
torch.Size([1, 512, 7, 7])
working on Slowpoke
working on 5 out of 19920
torch.Size([1, 512, 7, 7])
working on Suicune
working on 6 out of 19920
torch.Size([1, 512, 7, 7])
working on Donphan
working on 7 out of 19920
torch.Size([1, 512, 7, 7])
working on Sentret
working on 8 out of 19920
torch.Size([1, 512, 7, 7])
working on Wobbuffet
working on 9 out of 19920
torch.Size([1, 512, 7, 7])
working on Krabby
working on 10 out of 19920
torch.Size([1, 512, 7, 7])
working on Bulbasaur
working on 11 out of 19920
torch.Size([1, 512, 7, 7])
working on Charmeleon
working on 12 out of 19920
torch.Size([1, 512, 7, 7])
working on Gligar
worki

In [6]:
#Base VGGNet
class Pokemon_Classifier_VGGNet(nn.Module):
    def __init__(self, name = "Pokemon_Classifier_VGGNet"):
          self.name = "Pokemon_Classifier_VGGNet"
          super(Pokemon_Classifier_VGGNet, self).__init__()
          self.conv1 = nn.Conv2d(256, 50, 2) #in_channels, out_chanels, kernel_size
          self.conv2 = nn.Conv2d(50, 10, 2) #in_channels, out_chanels, kernel_size
          self.fc1 = nn.Linear(10*4*4, 300)
          self.fc2 = nn.Linear(300, len(pokemonList))

    def forward(self, x):
          x = F.relu(self.conv1(x))
          x = F.relu(self.conv2(x))
          x = x.view(-1, 10*4*4)
          print("Here",x.shape())
          x = F.relu(self.fc1(x))
          x = self.fc2(x)
          return x

In [7]:
def plotting(epochs, loss_chart, train_acc, val_acc):
    plt.title("Training Loss Curve")
    plt.plot(epochs, loss_chart, label="Train")
    plt.plot(epochs, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Accuracy Curve")
    plt.plot(epochs, train_acc, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()

In [8]:
use_cuda = True
def get_accuracy(model, test_loader):
    data = test_loader

    correct = 0
    total = 0
    for imgs, labels in data:
        #############################################
        #To Enable GPU Usage
        if use_cuda and torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()
        #############################################
        output = model(imgs)
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [10]:
use_cuda = True
def train(model, train_dataset, val_dataset, batch_size=128, learning_rate=0.001, num_epochs=20):

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                              num_workers=0, shuffle=True, drop_last = True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, 
                                              num_workers=1, shuffle=True, drop_last = True)
    train_acc = np.zeros(num_epochs)
    val_acc = np.zeros(num_epochs)
    loss_chart = np.zeros(num_epochs)
    epochs = []


    for imgs, labels in iter(train_loader):
      print(imgs.shape)
      print(pokemonList[labels[0]])
      print(imgs)
      break
      
    # training

    n = 0
    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_train_err = 0.0
        total_images = 0
        print(train_loader.dataset)
        for imgs, labels in iter(train_loader):
            print("h")
            if use_cuda == True:
              imgs, labels = imgs.cuda(), labels.cuda()

            out = model(imgs)             # forward pass
            print("After out")
            loss = criterion(out, labels) # compute the total loss
            print("After Loss")
            loss.backward()               # backward pass (compute parameter updates)
            print("After Back")
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch
        n += 1
        epochs.append(n)          
        
        # track accuracy
        train_acc[epoch] = get_accuracy(model, train_loader)
        val_acc[epoch] = get_accuracy(model, val_loader)
        loss_chart[epoch] = float(loss)/batch_size
        model_path = f"model_{model.name}_bs{batch_size}_lr{learning_rate}_epoch{epoch}"
        torch.save(model.state_dict(), model_path)


        print((f"Epoch {epoch+1}: Train accuracy = {train_acc[epoch]}"))

    epochs = np.arange(1, num_epochs + 1)

    plotting(epochs,loss_chart,train_acc, val_acc)

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))
    
    return train_acc, epochs

train_path = "DataSet/VGG2_Gen2_NewAug_V3/Training"
data_transforms = transforms.Compose([
                    transforms.Resize((224,224)),
                    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
train_dataset_new = torchvision.datasets.DatasetFolder(train_path, loader=torch.load, extensions=('.tensor'), transform=data_transforms)
val_path = "DataSet/VGG2_Gen2_NewAug_V3/Validation"
model = Pokemon_Classifier_VGGNet()
if use_cuda and torch.cuda.is_available():
  model.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')


train(model, train_dataset_new, val_path, batch_size = 1, learning_rate = 0.001, num_epochs=10)

FileNotFoundError: Found no valid file for the classes Farfetch_d. Supported extensions are: .tensor